In [ ]:
import scipy
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
truth = pd.Series(np.fromfile("../../pzip/data/icon.pl.u.f32.little.4x47x351x901_3.raw", dtype='uint32'))
truth.name = "truth"

In [ ]:
prediction = truth.shift()
prediction[0] = 0
prediction.name = "prediction"

In [ ]:
df = pd.DataFrame(truth).astype('uint32')
df['prediction'] = prediction.astype('uint32')

In [ ]:
df.tail()

In [ ]:
def _lzc(t):
    if t==0:
        return 32
    return 32 - len(np.binary_repr(t))
lzc = np.frompyfunc(_lzc, 1, 1)
npxor = np.frompyfunc(np.bitwise_xor, 1, 1)

In [ ]:
df['xor'] = npxor(df['truth'].values,df['prediction'].values)

In [ ]:
df['diff'] = df.loc[:,['prediction','truth']].max(axis=1) - df.loc[:,['prediction','truth']].min(axis=1)

In [ ]:
df['truth_bigger'] = df['truth'] >= df['prediction']

In [ ]:
df['xor_lzc'] = lzc(df['xor'])

In [ ]:
df.head()

In [ ]:
def _truth_bigger_or_same(prediction, lzc):
    if lzc == 32:
        # Truth is same as prediction
        return True
    value = 1 << 32 - lzc - 1
    if prediction & value > 0:
        return False
    else:
        return True
truth_bigger_or_same = np.frompyfunc(_truth_bigger, 2, 1)

In [ ]:
df['truth_bigger_lzc'] = truth_bigger_or_same(df['prediction'], df['xor_lzc'])

In [ ]:
# # ix = -3532
# # ix = -142
# ix = -24
# t = np.binary_repr(df['truth'].iloc[ix], 32)
# p = np.binary_repr(df['prediction'].iloc[ix], 32)
# l = np.binary_repr(1 << 32 - df['xor_lzc'].iloc[ix] - 1, 32)
# o = np.binary_repr(df['prediction'].iloc[ix] & 1 << 32 - df['xor_lzc'].iloc[ix] - 1, 32)
# print(t,p,l,o, sep='\n')

In [ ]:
np.array_equal(df['truth_bigger_lzc'], df['truth_bigger'])